In [1]:
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor

from evidently import ColumnMapping
from evidently.dashboard import Dashboard
from evidently.tabs import RegressionPerformanceTab

## Prepare data

In [2]:
dataset = fetch_california_housing(as_frame=True)
df = dataset.frame

In [3]:
target = 'MedHouseVal'
prediction = 'prediction'

numerical_features = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup',
                      'Latitude', 'Longitude']
categorical_features = []
features = numerical_features 

In [4]:
ref_data = df[:15000]
prod_data = df[15000:]

In [5]:
model = RandomForestRegressor(random_state=0)

In [6]:
model.fit(ref_data[features], ref_data[target])

RandomForestRegressor(random_state=0)

In [7]:
ref_data['prediction']  = model.predict(ref_data[features])
prod_data['prediction'] = model.predict(prod_data[features])

## Widget prototype

In [8]:
import plotly.figure_factory as ff

hist_data = [ref_data[target], prod_data[target]]

group_labels = ['reference', 'production']

colors = ['#333F44', '#37AA9C'] 

fig = ff.create_distplot(hist_data, group_labels, colors=colors, show_hist=False, show_rug=False)

fig.update_layout(   
    xaxis_title="Value", 
    yaxis_title="Share",
    xaxis=dict(
        showticklabels=True
    ),
    yaxis=dict(
        showticklabels=True
    ),
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1.02,
        xanchor="right",
        x=1
    ),
)

## Report with your own widget

In [8]:
from my_widgets.target_distribution_widget import TargetDistributionWidget

In [9]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = 'prediction'
column_mapping.numerical_features = numerical_features

In [10]:
dashboard = Dashboard(tabs=[RegressionPerformanceTab(include_widgets=[
    'Current: Predicted vs Actual',
    TargetDistributionWidget('Target distribution')
])])

In [11]:
dashboard.calculate(ref_data.sample(1000, random_state=0), 
                    prod_data.sample(1000, random_state=0), 
                    column_mapping=column_mapping)

In [12]:
dashboard.show()

## You own dashboard

In [13]:
from my_tabs.my_simple_tab import MySimpleTab

In [14]:
dashboard = Dashboard(tabs=[MySimpleTab()])

In [15]:
dashboard.calculate(ref_data.sample(1000, random_state=0), 
                    prod_data.sample(1000, random_state=0), 
                    column_mapping=column_mapping)

In [16]:
dashboard.show()